### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### Importing Dataframes

In [2]:
df= pd.read_csv('..\Original Data\customers.csv')

In [3]:
ords_prods_merge = pd.read_pickle('..\Prepared Data\orders_products_merged_updated_again_2.pkl')

### Step 4 (Wrangling the data)

In [6]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
# Renaming columns
df = df.rename(columns={'First Name':'First_name', 'Surnam':'Last_name', 'STATE':'State'})

In [8]:
df.head()

,user_id,First_name,Last_name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
df.shape

(206209, 10)

### Step 5 (Consistancy Check)

In [10]:
# Describe function check
df.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [11]:
# Data type check
df.dtypes

user_id          int64
First_name      object
Last_name       object
Gender          object
State           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [12]:
# Missing values check
df.isnull().sum()

user_id             0
First_name      11259
Last_name           0
Gender              0
State               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [13]:
# Checking missing columns
df.isnull().any()

user_id         False
First_name       True
Last_name       False
Gender          False
State           False
Age             False
date_joined     False
n_dependants    False
fam_status      False
income          False
dtype: bool

In [14]:
# Duplicates check
df_dups = df[df.duplicated()]

In [15]:
df_dups

,user_id,First_name,Last_name,Gender,State,Age,date_joined,n_dependants,fam_status,income


In [16]:
# Checking for mixed data
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

First_name


In [5]:
# Fixing "first_name"
df['First_name'] = df['First_name'].astype('str')

In [18]:
# Double checking for mixed data
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

In [19]:
df.isnull().sum()

user_id         0
First_name      0
Last_name       0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

### Step 6 (combining Data)

In [20]:
# Checking dataframe
ords_prods_merge.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_ordering,prior_ordering_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer


In [21]:
# Dropping columns
ords_prods_merge.drop(columns = ['eval_set'])

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_ordering,prior_ordering_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434207,1320836,202557,17,2,15,1.0,43553,2,1,Orange Energy Shots,...,both,Regularly busy,Regularly busy,Most orders,31,Regular customer,6.905655,Low spender,8.0,Frequent customer
32434208,31526,202557,18,5,11,3.0,43553,2,1,Orange Energy Shots,...,both,Regularly busy,Regularly busy,Most orders,31,Regular customer,6.905655,Low spender,8.0,Frequent customer
32434209,758936,203436,1,2,7,NaN,42338,4,0,"Zucchini Chips, Pesto",...,both,Regularly busy,Regularly busy,Average orders,3,New customer,7.631579,Low spender,15.0,Regular customer
32434210,2745165,203436,2,3,5,15.0,42338,16,1,"Zucchini Chips, Pesto",...,both,Regularly busy,Least busy days,Fewest orders,3,New customer,7.631579,Low spender,15.0,Regular customer


In [6]:
ords_prods_merge=ords_prods_merge.drop(columns = ['eval_set'])

In [16]:
ords_prods_merge.dtypes

order_id                    int64
user_id                     int64
order_number                int64
order_day_of_week           int64
time_ordered                int64
days_since_prior_order    float64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
product_name               object
aisle_id                    int64
department_id               int64
prices                    float64
busiest_day                object
Busiest_days               object
busiest_period_of_day      object
max_order                   int64
loyalty_flag               object
mean_order                float64
spending_flag              object
median_ordering           float64
prior_ordering_flag        object
dtype: object

In [7]:
ords_prods_merge=ords_prods_merge.drop(columns = ['_merge'])

In [17]:
ords_prods_merge.head(5)

,order_id,user_id,order_number,order_day_of_week,time_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_ordering,prior_ordering_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer


In [8]:
ords_prods_merge =ords_prods_merge.rename(columns={'order_dow':'order_day_of_week', 'order_hour_of_day':'time_ordered'})

In [18]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,time_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_ordering,prior_ordering_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,non-frequent customer


In [19]:
ords_prods_merge.dtypes

order_id                    int64
user_id                     int64
order_number                int64
order_day_of_week           int64
time_ordered                int64
days_since_prior_order    float64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
product_name               object
aisle_id                    int64
department_id               int64
prices                    float64
busiest_day                object
Busiest_days               object
busiest_period_of_day      object
max_order                   int64
loyalty_flag               object
mean_order                float64
spending_flag              object
median_ordering           float64
prior_ordering_flag        object
dtype: object

In [9]:
# Merging dataframes
df_combined = ords_prods_merge.merge (df, on='user_id',indicator = True)

In [11]:
#Checking merged dataframes
df_combined.head()

,order_id,user_id,order_number,order_day_of_week,time_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First_name,Last_name,Gender,State,Age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [12]:
df_combined.shape

(32434212, 32)

In [13]:
df_combined.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
order_day_of_week            int64
time_ordered                 int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
busiest_day                 object
Busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
mean_order                 float64
spending_flag               object
median_ordering            float64
prior_ordering_flag         object
First_name                  object
Last_name                   object
Gender                      object
State                       object
Age                          int64
date_joined                 object
n_dependants        

### Exporting Dataframe as Pickle

In [15]:
df_combined.to_pickle('..\Prepared Data\ords_prods_customers_merged.pkl')